## Clean and Combine Public Restroom Data

In [54]:
import pandas as pd
import numpy as np
from geopy.geocoders import GoogleV3
import geopy.distance
import googlemaps

### Load Data

In [113]:
data1 = pd.read_csv("Directory_Of_Toilets_In_Public_Parks.csv") #https://data.cityofnewyork.us/Recreation/Directory-Of-Toilets-In-Public-Parks/hjae-yuav
data2 = pd.read_csv("dot_Automatic_Public_Toilets_20180918.csv") #https://data.cityofnewyork.us/dataset/Automatic-Public-Toilets/uzgy-xh4j

print(data1.columns, data2.columns)
print(data1.shape, data2.shape)

Index(['Name', 'Location', 'Open Year-Round', 'Handicap Accessible', 'Borough',
       'Comments'],
      dtype='object') Index(['the_geom', 'BoroCode', 'BoroName', 'BoroCD', 'CounDist', 'AssemDist',
       'StSenDist', 'CongDist', 'Site_Name', 'Street', 'NTAName', 'FEMAFldz',
       'FEMAFldT', 'HrcEvac'],
      dtype='object')
(616, 6) (5, 14)


### Add/Format Lat and Long coords for restroom location

In [109]:
API = "..."
geolocator = GoogleV3(api_key=API)

def getlnglat(address):
    location = geolocator.geocode(address)
    if location != None:
        return location.longitude, location.latitude
    else:
        print("DUD:", address)
        return np.nan, np.nan

longitude = []
latitude = []
for i, _ in enumerate(data1.Location):
    if data1.Location[i]:
        address = "{}, {}, {}, New York".format(data1.Name[i], data1.Location[i], data1.Borough[i])
    else:
        address = "{}, {}, New York".format(data1.Name[i], data1.Borough[i])
    lng, lat = getlnglat(address)
    latitude.append(lng)
    longitude.append(lat)

In [114]:
data1["Latitude"] = latitude
data1["Longitude"] = longitude
data1.head()

,Name,Location,Open Year-Round,Handicap Accessible,Borough,Comments,Latitude,Longitude
0,100% Playground,"Glenwood Road, East 100 & East 101 streets",Yes,NaN,Brooklyn,NaN,-73.899055,40.646590
1,174th Street Playground,"East 174 Street, Stratford Avenue, Bronx River...",Yes,NaN,Bronx,NaN,-73.887809,40.837445
2,227 Street Playground,Bronx Boulevard between East 226 and East 228 ...,Yes,Yes,Bronx,NaN,-73.864220,40.890810
3,Abe Lincoln,"East 135 Street, between Madison & 5 avenues",Yes,NaN,Manhattan,NaN,-73.937215,40.812297
4,Abigail Playground,"East 156 Street, Tinton Avenue",No,NaN,Bronx,NaN,-73.904843,40.817328


In [139]:
data2_lat = []
data2_lon = []

for i, _ in enumerate(data2.the_geom):
    lat = data2.the_geom[i].split()[1][1:]
    lon = data2.the_geom[i].split()[2][:-1]
    data2_lat.append(float(lat))
    data2_lon.append(float(lon))

data2["Latitude"] = data2_lat
data2["Longitude"] = data2_lon
data2.head()


,the_geom,BoroCode,BoroName,BoroCD,CounDist,AssemDist,StSenDist,CongDist,Site_Name,Street,NTAName,FEMAFldz,FEMAFldT,HrcEvac,Latitude,Longitude
0,POINT (-73.96907239517027 40.672228846277484),3,Brooklyn,355,39,44,20,10,Grand Army Plaza,FLATBUSH AVENUE,Prospect Park,X,AREA OF MINIMAL FLOOD HAZARD,NaN,-73.969072,40.672229
1,POINT (-73.86263617948991 40.74964121058453),4,Queens,404,21,34,13,14,Corona Plaza,ROOSEVELT AVENUE,Corona,X,AREA OF MINIMAL FLOOD HAZARD,NaN,-73.862636,40.749641
2,POINT (-73.98784339765545 40.74130070685076),1,Manhattan,105,2,74,28,12,Madison Square Park,MADISON AVENUE,Midtown South-Flatiron-Union Square,X,AREA OF MINIMAL FLOOD HAZARD,NaN,-73.987843,40.741301
3,POINT (-73.93799584577202 40.84592127411525),1,Manhattan,112,10,72,31,13,Plaza de Las Americas,WEST 175 STREET,Washington Heights (South),X,AREA OF MINIMAL FLOOD HAZARD,NaN,-73.937996,40.845921
4,POINT (-73.89115785635144 40.860825986041235),2,Bronx,206,15,78,33,15,Fordham Plaza,PARK AVENUE,Belmont,X,AREA OF MINIMAL FLOOD HAZARD,NaN,-73.891158,40.860826


### Clean up Data

#### Remove Unecessary Columns

In [140]:
#drop unnecessary columns from both dataframes, make sure headings are the same
data1 = data1.drop(columns=['Comments'])
data2 = data2.drop(columns=['the_geom', 'BoroCode', 'BoroCD', 'CounDist', 'AssemDist',
       'StSenDist', 'CongDist', 'NTAName', 'FEMAFldz',
       'FEMAFldT', 'HrcEvac'])

In [141]:
data1.head()

,Name,Location,Open Year-Round,Handicap Accessible,Borough,Latitude,Longitude
0,100% Playground,"Glenwood Road, East 100 & East 101 streets",Yes,NaN,Brooklyn,-73.899055,40.646590
1,174th Street Playground,"East 174 Street, Stratford Avenue, Bronx River...",Yes,NaN,Bronx,-73.887809,40.837445
2,227 Street Playground,Bronx Boulevard between East 226 and East 228 ...,Yes,Yes,Bronx,-73.864220,40.890810
3,Abe Lincoln,"East 135 Street, between Madison & 5 avenues",Yes,NaN,Manhattan,-73.937215,40.812297
4,Abigail Playground,"East 156 Street, Tinton Avenue",No,NaN,Bronx,-73.904843,40.817328


In [142]:
data2.head()

,BoroName,Site_Name,Street,Latitude,Longitude
0,Brooklyn,Grand Army Plaza,FLATBUSH AVENUE,-73.969072,40.672229
1,Queens,Corona Plaza,ROOSEVELT AVENUE,-73.862636,40.749641
2,Manhattan,Madison Square Park,MADISON AVENUE,-73.987843,40.741301
3,Manhattan,Plaza de Las Americas,WEST 175 STREET,-73.937996,40.845921
4,Bronx,Fordham Plaza,PARK AVENUE,-73.891158,40.860826


#### Fix capitalization

In [143]:
data2['Street'] = [s.capitalize() for s in data2['Street']]

In [144]:
data2.head()

,BoroName,Site_Name,Street,Latitude,Longitude
0,Brooklyn,Grand Army Plaza,Flatbush avenue,-73.969072,40.672229
1,Queens,Corona Plaza,Roosevelt avenue,-73.862636,40.749641
2,Manhattan,Madison Square Park,Madison avenue,-73.987843,40.741301
3,Manhattan,Plaza de Las Americas,West 175 street,-73.937996,40.845921
4,Bronx,Fordham Plaza,Park avenue,-73.891158,40.860826


#### Fix Headings

In [146]:
data2.rename(columns = {'BoroName':'Borough', 'Site_Name':'Name', 'Street':'Location'}, inplace = True)
data2.head()


,Borough,Name,Location,Latitude,Longitude
0,Brooklyn,Grand Army Plaza,Flatbush avenue,-73.969072,40.672229
1,Queens,Corona Plaza,Roosevelt avenue,-73.862636,40.749641
2,Manhattan,Madison Square Park,Madison avenue,-73.987843,40.741301
3,Manhattan,Plaza de Las Americas,West 175 street,-73.937996,40.845921
4,Bronx,Fordham Plaza,Park avenue,-73.891158,40.860826


#### Combine Data, fix NaN values

In [147]:
data = pd.concat([data1, data2], ignore_index=True)
print(data1.shape[0] + data2.shape[0])
print(data.shape)

621
(621, 7)


In [148]:
data["Handicap Accessible"] = data["Handicap Accessible"].fillna("Unknown")
data["Open Year-Round"] = data["Open Year-Round"].fillna("Unknown")
data["Location"] = data["Location"].fillna("-")


#### Check Datatypes

In [149]:
# Check datatypes
data = data.convert_dtypes()
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621 entries, 0 to 620
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 621 non-null    string 
 1   Location             621 non-null    string 
 2   Open Year-Round      621 non-null    string 
 3   Handicap Accessible  621 non-null    string 
 4   Borough              621 non-null    string 
 5   Latitude             621 non-null    Float64
 6   Longitude            621 non-null    Float64
dtypes: Float64(2), string(5)
memory usage: 35.3 KB
None


,Name,Location,Open Year-Round,Handicap Accessible,Borough,Latitude,Longitude
0,100% Playground,"Glenwood Road, East 100 & East 101 streets",Yes,Unknown,Brooklyn,-73.899055,40.64659
1,174th Street Playground,"East 174 Street, Stratford Avenue, Bronx River...",Yes,Unknown,Bronx,-73.887809,40.837445
2,227 Street Playground,Bronx Boulevard between East 226 and East 228 ...,Yes,Yes,Bronx,-73.86422,40.89081
3,Abe Lincoln,"East 135 Street, between Madison & 5 avenues",Yes,Unknown,Manhattan,-73.937215,40.812297
4,Abigail Playground,"East 156 Street, Tinton Avenue",No,Unknown,Bronx,-73.904843,40.817328


### Save Data as csv

In [150]:
data.to_csv("nyc-public-restrooms.csv", index=False)